##Laboratorio Sobre el Perceptrón



Seleccionamos el dataset de [cancer de mama](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_breast_cancer.html?highlight=breast%20cancer ) de sklearn que contiene 569 ejemplos y dos clases para su clasificación, definiendo si el tumor es maligno o benigno.

In [1]:
from sklearn import datasets
import pandas as pd
import numpy as np
#we import the data
cancer = datasets.load_breast_cancer(return_X_y=True, as_frame=True)
#determine the number of registers
number_of_registers = len(cancer[0])
#determine the number of training cases
training_cases = int(number_of_registers*0.8)
#and thus, the number of test registers
test_cases = number_of_registers - training_cases
#we select  indexes at random, as many of the training set size we desire
selection_training = np.random.choice(number_of_registers,training_cases)
#we take the data at these indexes
training_set = cancer[0].iloc[selection_training].values, np.where(cancer[1][selection_training]==1, 1, -1)
#we take the remainder indexes
test_selection = np.array([i for i in range(number_of_registers) if i not in selection_training])
#and take those points of data in order to create a test set
test_set = cancer[0].iloc[test_selection].values, np.where(cancer[1][test_selection]==1, 1, -1)

Train y test con dos variables

In [25]:
train_2 = cancer[0].iloc[selection_training][['mean texture','mean symmetry']].values, np.where(cancer[1][selection_training]==1, 1, -1)
test_2 = cancer[0].iloc[test_selection][['mean texture','mean symmetry']].values, np.where(cancer[1][test_selection]==1, 1, -1)

In [2]:
print(cancer[0].columns)

Index(['mean radius', 'mean texture', 'mean perimeter', 'mean area',
       'mean smoothness', 'mean compactness', 'mean concavity',
       'mean concave points', 'mean symmetry', 'mean fractal dimension',
       'radius error', 'texture error', 'perimeter error', 'area error',
       'smoothness error', 'compactness error', 'concavity error',
       'concave points error', 'symmetry error', 'fractal dimension error',
       'worst radius', 'worst texture', 'worst perimeter', 'worst area',
       'worst smoothness', 'worst compactness', 'worst concavity',
       'worst concave points', 'worst symmetry', 'worst fractal dimension'],
      dtype='object')


In [3]:
#we create our perceptron
class Perceptron:
    def __init__(self, eta=0.1, n_iter=10):
        self.eta = float(eta)
        self.n_iter = n_iter

    def train(self, X, y):
        # inicializar los pesos en 0
        self.w = np.zeros(len(X[1])+1)
        #vector de errores acumulados
        self.errors = []

        #ciclo de entrenamiento
        for i in range(self.n_iter):
            errors = 0
            for x_i, target in zip(X,y):
                #calcular el nuevo valor de los pesos
                delta_w = np.array((target - self.predict(x_i)) * self.eta)
                #actualizar el valor de los pesos
                self.w[1:] += delta_w * x_i
                #actualizar el valor del bias
                self.w[0] += delta_w
                if (delta_w!=0):
                    errors += 1
            self.errors.append(errors)
 

    def predict(self, X):
        #combinacion lineal, w[0] = bias
        v = np.dot( X, self.w [1:]) + self.w[0]
        #funcion de activación 
        if v > 0.0:
            return 1
        return -1

Entrenamiento y predicción con todas las variables del dataset

In [4]:
p = Perceptron( n_iter = 5000)
p.train(training_set[0], training_set[1])
aciertos = list()
for i, j in zip(test_set[0],test_set[1]):
  aciertos.append(p.predict(i) == j)
print(sum(aciertos)/len(aciertos))

0.9090909090909091


Entrenamiento y predicción con 2 variables del dataset

In [28]:
p = Perceptron( n_iter = 5000)
p.train(train_2[0], train_2[1])
aciertos = list()
for i, j in zip(test_2[0],test_2[1]):
  aciertos.append(p.predict(i) == j)
print(sum(aciertos)/len(aciertos))

0.7154150197628458


##Complejidad

La complejidad de este algoritmo depende principalmente de la cantidad de parámetros que usaremos y sus iteraciones. Sea $n$ el número de parámetros e $i$ el número de iteraciones.Para calcular delta_w es necesario ralizar tantas multiplicaciones como parámetros, el nuevo w requiere esta misma cantidad de multiplicaciones (como es stándar, no contamos las sumas). Esta operación se repite tantas veces como datos en el set de entrenamiento, llamemos a este número $m$. Por último, todo esto se repite tantas veces como iteraciones tenemos. Esto es: $O(imn)$. Si asumimos $i=n=m$ tenemos $O(n^3)$.

#### Separabilidad de los datos

Cuando se usan todas las variables es muy complejo definir la separabilidad de los datos. Sin embargo, por el índice que se obtuvo al evaluar las predicciones del perceptrón se puede afirmar que los datos deben ser separables.